In [1]:
# ============================================================
# 对比实验：没有DES控制器的LLM交互（Baseline）
# ============================================================
# 本代码展示在没有形式化控制的情况下，LLM可能产生的违规行为
# 对比 llm_control_tct.ipynb 中使用DES控制器的版本

print("="*60)
print("--- 对比实验：无控制器的LLM交互（Baseline）---")
print("--- 场景：技术支持机器人 ---")
print("="*60)
print("\n注意：此版本没有使用DES控制器，LLM可以自由选择任何动作")
print("      包括可能违反规范的行为（如：在收集信息前就升级问题）\n")

--- 对比实验：无控制器的LLM交互（Baseline）---
--- 场景：技术支持机器人 ---

注意：此版本没有使用DES控制器，LLM可以自由选择任何动作
      包括可能违反规范的行为（如：在收集信息前就升级问题）



In [2]:
# --- 定义状态和事件（与控制版本相同的Plant模型）---

# 状态定义
states = {
    0: "S0_Idle",
    1: "S1_ProblemReceived",
    2: "S2_InfoGathering",
    3: "S3_SolutionProposed",
    4: "S4_Escalated",
    5: "S5_Resolved"
}

# 所有可能的转移（包括违规的转移）
transitions = [
    (0, 'p_ask_problem', 1, 'c'),
    (1, 'p_ask_details', 2, 'c'),
    (1, 'r_is_confused', 1, 'u'),
    (2, 'r_provides_info', 3, 'u'),
    (2, 'r_is_confused', 2, 'u'),
    (3, 'r_confirms_solved', 5, 'u'),
    (3, 'r_requests_escalation', 4, 'u'),
    (3, 'r_is_confused', 3, 'u'),
    (1, 'p_escalate', 4, 'c'),  # 违规行为：在收集信息前就升级
    (3, 'p_escalate', 4, 'c'),
    (4, 'r_is_confused', 4, 'u'),
    (4, 'p_end_session', 0, 'c'),
    (5, 'r_is_confused', 5, 'u'),
    (5, 'p_end_session', 0, 'c'),
]

# 构建转移字典，方便查询
transition_dict = {}
for src, event, dst, ctrl in transitions:
    if src not in transition_dict:
        transition_dict[src] = []
    transition_dict[src].append((event, dst, ctrl))

print("[1] Plant 模型定义完成（无控制器）")
print(f"    状态数: {len(states)}")
print(f"    转移数: {len(transitions)}")
print("    ⚠️  包含违规转移: (S1_ProblemReceived, p_escalate, S4_Escalated)")

[1] Plant 模型定义完成（无控制器）
    状态数: 6
    转移数: 14
    ⚠️  包含违规转移: (S1_ProblemReceived, p_escalate, S4_Escalated)


In [3]:
# --- 定义Prompt文本和模拟LLM ---

prompt_texts = {
    'p_ask_problem': "Hello! I'm a support bot. Please describe your problem.",
    'p_ask_details': "Thank you. Can you provide more details?",
    'p_escalate': "I will escalate this issue to a human agent.",
    'p_end_session': "Session is ending. Goodbye."
}

def mock_llm_uncontrolled(prompt, current_state_name):
    """
    模拟LLM响应（无控制版本）
    注意：这里的响应可能导致违规行为
    """
    print(f"[LLM Input] Prompt: '{prompt}' (from state: {current_state_name})")
    
    if "describe your problem" in prompt:
        return "My computer won't turn on.", "r_is_confused"
    elif "more details" in prompt:
        return "I've already checked the power cable.", "r_provides_info"
    elif "ending" in prompt or "Goodbye" in prompt:
        return "Thank you!", "r_confirms_solved"
    else:
        return "I am confused.", "r_is_confused"

print("[2] Prompt文本和模拟LLM定义完成")

[2] Prompt文本和模拟LLM定义完成


In [4]:
# --- 无控制器的交互模拟 ---

print("\n" + "="*60)
print("--- 开始无控制器的交互模拟 ---")
print("="*60)

current_state = 0
violation_detected = False
info_gathered = False  # 标记是否已收集信息
step_count = 0

while True:
    step_count += 1
    current_state_name = states[current_state]
    
    print(f"\n{'='*60}")
    print(f"Step {step_count}: 当前状态 = {current_state_name} (State {current_state})")
    print(f"{'='*60}")
    
    # 获取当前状态的所有可能转移
    if current_state not in transition_dict:
        print("[Session] 到达终端状态，会话结束。")
        break
    
    available_transitions = transition_dict[current_state]
    
    # 筛选可控事件（Prompts）
    controllable_actions = [(e, d) for e, d, c in available_transitions if c == 'c' and e in prompt_texts]
    
    if not controllable_actions:
        print("[Info] 当前状态没有可执行的Prompt，等待不可控事件...")
        uncontrollable = [(e, d) for e, d, c in available_transitions if c == 'u']
        if not uncontrollable:
            print("[Session] 无可用转移，会话结束。")
            break
        event, next_state = uncontrollable[0]
        print(f"[Event] 自动触发不可控事件: {event}")
        current_state = next_state
        continue
    
    # 显示所有可用的动作（包括违规动作）
    print("\n--- 可用的动作 (Prompts) ---")
    for i, (event, _) in enumerate(controllable_actions):
        warning = ""
        # 检测违规动作
        if event == 'p_escalate' and not info_gathered:
            warning = " ⚠️  [违规动作：未收集信息就升级]"
        print(f"  {i+1}: {prompt_texts[event]} ({event}){warning}")
    
    # 自动选择（模拟无控制的决策）
    # 为了展示违规行为，我们在某些情况下故意选择违规动作
    try:
        choice = int(input("\n输入选项编号 (或直接回车自动选择): ").strip() or "1") - 1
        if choice < 0 or choice >= len(controllable_actions):
            choice = 0
    except (ValueError, EOFError):
        choice = 0
    
    chosen_event, next_state = controllable_actions[choice]
    
    # 检测违规
    if chosen_event == 'p_escalate' and not info_gathered:
        print(f"\n{'!'*60}")
        print("⚠️  违规行为检测！")
        print("    规范要求：在事件 'r_provides_info' 发生之前，")
        print("             事件 'p_escalate' 不能发生")
        print("    当前行为：在未收集用户信息的情况下直接升级问题")
        print(f"{'!'*60}\n")
        violation_detected = True
    
    print(f"\n[Action] 执行动作: {chosen_event}")
    current_state = next_state
    
    # 如果是结束会话，直接退出
    if chosen_event == 'p_end_session':
        print("\n[Session] 会话结束。")
        break
    
    # 调用LLM获取响应
    llm_response_text, llm_response_event = mock_llm_uncontrolled(
        prompt_texts[chosen_event], states[current_state]
    )
    print(f"    [LLM Output] Response: '{llm_response_text}' (分类为: '{llm_response_event}')")
    
    # 记录是否收集了信息
    if llm_response_event == 'r_provides_info':
        info_gathered = True
        print("    ✓ 信息收集完成")
    
    # 根据LLM响应更新状态
    next_state_found = False
    for event, dst, ctrl in transition_dict.get(current_state, []):
        if event == llm_response_event:
            print(f"[Event] 发生事件: {llm_response_event}")
            current_state = dst
            next_state_found = True
            break
    
    if not next_state_found:
        print(f"\n⚠️  警告：在状态 {current_state} 发生意外事件 '{llm_response_event}'")
        print("[Session] 会话异常终止。")
        break
    
    # 限制最大步数，防止无限循环
    if step_count >= 10:
        print("\n[Session] 达到最大步数限制，会话结束。")
        break

print("\n" + "="*60)
print("--- 模拟结束 ---")
print("="*60)

if violation_detected:
    print("\n❌ 结果：检测到违规行为！")
    print("   在没有DES控制器的情况下，LLM可能执行违反规范的动作。")
else:
    print("\n✓ 结果：本次交互未检测到违规行为。")
    print("  （但这不意味着系统总是安全的，只是本次模拟中未触发违规）")


--- 开始无控制器的交互模拟 ---

Step 1: 当前状态 = S0_Idle (State 0)

--- 可用的动作 (Prompts) ---
  1: Hello! I'm a support bot. Please describe your problem. (p_ask_problem)

[Action] 执行动作: p_ask_problem
[LLM Input] Prompt: 'Hello! I'm a support bot. Please describe your problem.' (from state: S1_ProblemReceived)
    [LLM Output] Response: 'My computer won't turn on.' (分类为: 'r_is_confused')
[Event] 发生事件: r_is_confused

Step 2: 当前状态 = S1_ProblemReceived (State 1)

--- 可用的动作 (Prompts) ---
  1: Thank you. Can you provide more details? (p_ask_details)
  2: I will escalate this issue to a human agent. (p_escalate) ⚠️  [违规动作：未收集信息就升级]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
⚠️  违规行为检测！
    规范要求：在事件 'r_provides_info' 发生之前，
             事件 'p_escalate' 不能发生
    当前行为：在未收集用户信息的情况下直接升级问题
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


[Action] 执行动作: p_escalate
[LLM Input] Prompt: 'I will escalate this issue to a human agent.' (from state: S4_Escalated)
    [LLM Output] Respons

In [5]:
# --- 对比总结 ---

print("\n" + "="*60)
print("--- 对比总结：有控制器 vs 无控制器 ---")
print("="*60)

print("""
┌─────────────────────────────────────────────────────────────┐
│                    对比维度                                  │
├─────────────────────────────────────────────────────────────┤
│ 1. 状态空间                                                  │
│    - 无控制器：所有可能的状态和转移都可达                    │
│    - 有控制器：只有符合规范的状态和转移可达                  │
│                                                              │
│ 2. 动作选择                                                  │
│    - 无控制器：LLM可以自由选择任何可控事件                   │
│    - 有控制器：只能选择控制器允许的可控事件                  │
│                                                              │
│ 3. 规范保证                                                  │
│    - 无控制器：无法保证规范被遵守，可能违规                  │
│    - 有控制器：形式化保证规范被遵守，不会违规                │
│                                                              │
│ 4. 关键差异示例                                              │
│    规范："在 r_provides_info 之前，p_escalate 不能发生"      │
│                                                              │
│    - 无控制器：在 S1_ProblemReceived 状态下，                │
│                LLM可以选择 p_escalate（违规）                │
│                                                              │
│    - 有控制器：在 S1_ProblemReceived 状态下，                │
│                p_escalate 不在可选动作中（被禁止）           │
│                                                              │
│ 5. 实际应用价值                                              │
│    - 无控制器：适合探索性任务，但不适合关键业务              │
│    - 有控制器：适合需要严格遵守规范的关键业务场景            │
│                （如客服、医疗、金融等）                       │
└─────────────────────────────────────────────────────────────┘
""")

print("\n💡 关键洞察：")
print("   DES控制器通过形式化方法，在运行时动态限制LLM的动作空间，")
print("   确保LLM的行为始终符合预定义的规范，从而提高系统的可靠性")
print("   和安全性。这是传统Prompt工程难以实现的强保证。")

print("\n📊 建议的对比实验：")
print("   1. 运行本文件（llm_tct.ipynb）- 观察无控制器的行为")
print("   2. 运行 llm_control_tct.ipynb - 观察有控制器的行为")
print("   3. 对比两者在相同场景下的动作选择和规范遵守情况")
print("   4. 特别关注：在 S1_ProblemReceived 状态下是否允许 p_escalate")


--- 对比总结：有控制器 vs 无控制器 ---

┌─────────────────────────────────────────────────────────────┐
│                    对比维度                                  │
├─────────────────────────────────────────────────────────────┤
│ 1. 状态空间                                                  │
│    - 无控制器：所有可能的状态和转移都可达                    │
│    - 有控制器：只有符合规范的状态和转移可达                  │
│                                                              │
│ 2. 动作选择                                                  │
│    - 无控制器：LLM可以自由选择任何可控事件                   │
│    - 有控制器：只能选择控制器允许的可控事件                  │
│                                                              │
│ 3. 规范保证                                                  │
│    - 无控制器：无法保证规范被遵守，可能违规                  │
│    - 有控制器：形式化保证规范被遵守，不会违规                │
│                                                              │
│ 4. 关键差异示例                                              │
│    规范："在 r_provides_info 之前，p_escalate 不能发生"      │
│             